In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re  # Importamos el módulo 're' para usar expresiones regulares
import unicodedata

### Analisis de tabla purchase_prices

In [199]:
purchase_prices = pd.read_csv('2017PurchasePricesDec.csv')
purchase_prices.head()

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


#### Identificar problemas en tipos de datos

In [200]:
# Revisar los tipos de datos para cada tabla
purchase_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12261 entries, 0 to 12260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12261 non-null  int64  
 1   Description     12260 non-null  object 
 2   Price           12261 non-null  float64
 3   Size            12260 non-null  object 
 4   Volume          12260 non-null  object 
 5   Classification  12261 non-null  int64  
 6   PurchasePrice   12261 non-null  float64
 7   VendorNumber    12261 non-null  int64  
 8   VendorName      12261 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 862.2+ KB


#### Modificar tipo de dato

In [201]:
# Función para unificar tipos de datos en un DataFrame
def unify_column_types(df):
    # Convertir columnas numéricas a tipos consistentes
    for col in df.select_dtypes(include=['number']).columns:
        if df[col].dtype == 'float64' and df[col].dropna().apply(float.is_integer).all():
            # Convertir columnas flotantes que tienen solo valores enteros a int64
            df[col] = df[col].astype('int64')
        elif df[col].dtype == 'int64':
            continue  # Mantener columnas que ya son int64
        else:
            df[col] = df[col].astype('float64')  # Asegurar columnas numéricas flotantes

    # Convertir columnas de texto a string
    string_columns = df.select_dtypes(include=['object']).columns
    df[string_columns] = df[string_columns].astype('string')

    return df

# Aplicar la función al DataFrame
purchase_prices = unify_column_types(purchase_prices)

# Verificar el resultado final
print("Tipos de datos después de unificar:")
print(purchase_prices.dtypes)

# Mostrar las primeras filas
print(purchase_prices.head())


Tipos de datos después de unificar:
Brand                      int64
Description       string[python]
Price                    float64
Size              string[python]
Volume            string[python]
Classification             int64
PurchasePrice            float64
VendorNumber               int64
VendorName        string[python]
dtype: object
   Brand                  Description  Price   Size Volume  Classification  \
0     58  Gekkeikan Black & Gold Sake  12.99  750mL    750               1   
1     62     Herradura Silver Tequila  36.99  750mL    750               1   
2     63   Herradura Reposado Tequila  38.99  750mL    750               1   
3     72         No. 3 London Dry Gin  34.99  750mL    750               1   
4     75    Three Olives Tomato Vodka  14.99  750mL    750               1   

   PurchasePrice  VendorNumber                   VendorName  
0           9.28          8320  SHAW ROSS INT L IMP LTD      
1          28.67          1128  BROWN-FORMAN CORP           

In [202]:

# Función para limpiar strings
def clean_string(s):
    if pd.isnull(s):  # Si es nulo, lo devuelve tal cual
        return s
    # Eliminar acentos y normalizar el texto
    s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('utf-8')
    # Convertir a minúsculas, quitar espacios al inicio/final y reemplazar múltiples espacios
    return ' '.join(s.strip().lower().split())

# Convertir columnas de tipo object a string y limpiarlas
purchase_prices = purchase_prices.convert_dtypes()  # Cambiar a tipos más apropiados
string_columns = purchase_prices.select_dtypes(include=['string', 'object']).columns

for col in string_columns:
    purchase_prices[col] = purchase_prices[col].apply(clean_string)

# Reemplazar valores "Unknown" y pd.NA en Volume por NaN y convertirla a float
purchase_prices['Volume'] = purchase_prices['Volume'].replace(["unknown", pd.NA], np.nan).astype(float)

# Asegurarse de que todas las columnas de tipo object se conviertan a string
string_columns = purchase_prices.select_dtypes(include=['object']).columns
purchase_prices[string_columns] = purchase_prices[string_columns].astype('string')

# Verificar los resultados finales
print("Primeras filas después de limpiar las columnas de tipo string:")
print(purchase_prices.head())
print("Tipos de datos después de la conversión a string:")
print(purchase_prices.dtypes)


Primeras filas después de limpiar las columnas de tipo string:
   Brand                  Description  Price   Size  Volume  Classification  \
0     58  gekkeikan black & gold sake  12.99  750ml   750.0               1   
1     62     herradura silver tequila  36.99  750ml   750.0               1   
2     63   herradura reposado tequila  38.99  750ml   750.0               1   
3     72         no. 3 london dry gin  34.99  750ml   750.0               1   
4     75    three olives tomato vodka  14.99  750ml   750.0               1   

   PurchasePrice  VendorNumber                  VendorName  
0           9.28          8320     shaw ross int l imp ltd  
1          28.67          1128           brown-forman corp  
2          30.46          1128           brown-forman corp  
3          26.11          9165  ultra beverage company llp  
4          10.94          7245        proximo spirits inc.  
Tipos de datos después de la conversión a string:
Brand                      Int64
Description  

#### Analisis de columna 'Size' 

In [203]:
# Verificar valores únicos en Size
print(purchase_prices['Size'].unique())

<StringArray>
[      '750ml',      '1000ml',      '1750ml',        '50ml',       '375ml',
  '100ml 4 pk',   '50ml 5 pk',       '100ml',       '200ml',   '50ml 4 pk',
   '50ml 3 pk',       '300ml',  '200ml 4 pk',     'unknown',  '750ml 2 pk',
  '250ml 4 pk',      '1500ml',      '3000ml',      '5000ml',      '4000ml',
  '187ml 4 pk',       '150ml',       '187ml',       '500ml',       '720ml',
       '650ml',  '200ml 3 pk',  '187ml 3 pk',       '330ml',       '250ml',
  '750ml + 3/',     '18000ml',       '180ml',  '750ml + 4/',      '6000ml',
 '1000ml 2 pk',  '750ml 12 p',  '750ml 6 pk',   '20ml 5 pk',          <NA>,
  '375ml 2 pk',     '20000ml',  '50ml 12 pk',  '750ml 3 pk',  '375ml 3 pk',
  '750ml + 1/',  '200ml 5 pk',     '162.5ml',       '400ml',      '1100ml',
       '600ml',     '19500ml',       '560ml',      '3750ml',     '750ml 3',
      '9000ml']
Length: 56, dtype: string


La mayoría de los valores están en mililitros (ml).
Algunos tienen sufijos como pk para indicar paquetes.
Otros incluyen valores desconocidos (Unknown) o formatos adicionales (+ 3/)

#### Transformacion de columna size para obtener mL totales, paquetes (No use la columna de volumen porque no me indica correctamente cuando hay paquetes) 

In [204]:
# Función para procesar la columna 'Size'
def parse_size(size):
    if pd.isnull(size):  # Verificar si el valor es nulo
        return {"size_ml": np.nan, "pack_count": np.nan}  # Retorna valores nulos si el campo es vacío
    
    size = size.strip().lower()  # Elimina espacios al principio/final y convierte a minúsculas

    #################################################################################################
    # Manejar paquetes, como "100mL 4 Pk"
    # Busca coincidencias con el patrón (\d+(?:\.\d+)?)(ml|l) (\d+) pk
    #################################################################################################
    match = re.match(r"(\d+(?:\.\d+)?)(ml|l) (\d+) pk", size)  # Buscar coincidencias con regex
    if match:
        unit_size = float(match.group(1))  # Tamaño individual
        if match.group(2) == 'l':  # Convertir litros a mililitros
            unit_size *= 1000
        pack_count = int(match.group(3))  # Número de botellas en el paquete
        return {"size_ml": unit_size, "pack_count": pack_count}

    #################################################################################################
    # Manejar tamaños con adicionales, como "750mL + 3/"
    #################################################################################################
    match = re.match(r"(\d+(?:\.\d+)?)(ml|l)(?: \+ (\d+)/)?", size)  # Buscar coincidencias con regex para valores con +
    if match:
        unit_size = float(match.group(1))  # Tamaño individual
        if match.group(2) == 'l':  # Convertir litros a mililitros
            unit_size *= 1000
        additional_count = int(match.group(3)) if match.group(3) else 0  # Ítems adicionales
        return {"size_ml": unit_size, "pack_count": 1 + additional_count}

    #################################################################################################
    # Manejar tamaños con número adicional separado por espacios, como "750mL  3"
    #################################################################################################
    match = re.match(r"(\d+(?:\.\d+)?)(ml|l)\s+(\d+)", size)  # Buscar coincidencias con espacios adicionales
    if match:
        unit_size = float(match.group(1))  # Tamaño individual
        if match.group(2) == 'l':  # Convertir litros a mililitros
            unit_size *= 1000
        pack_count = int(match.group(3))  # Número adicional
        return {"size_ml": unit_size, "pack_count": pack_count}

    #################################################################################################
    # Manejar tamaños individuales, como "750mL" o "162.5mL"
    #################################################################################################
    match = re.match(r"(\d+(?:\.\d+)?)(ml|l)", size)  # Busca tamaños individuales
    if match:
        unit_size = float(match.group(1))  # Tamaño individual
        if match.group(2) == 'l':  # Convertir litros a mililitros
            unit_size *= 1000
        return {"size_ml": unit_size, "pack_count": 1}

    #################################################################################################
    # Retornar valores nulos si el tamaño no coincide con ningún patrón conocido
    #################################################################################################
    return {"size_ml": np.nan, "pack_count": np.nan}

# Aplicar la función parse_size a la columna 'Size'
size_data = purchase_prices['Size'].apply(parse_size)

# Crear nuevas columnas en el DataFrame
purchase_prices['size_ml'] = size_data.apply(lambda x: x['size_ml'])
purchase_prices['pack_count'] = size_data.apply(lambda x: x['pack_count'])

# Verificar las primeras filas para confirmar resultados
print("Primeras filas del DataFrame después de procesar la columna 'Size':")
print(purchase_prices[['Size', 'size_ml', 'pack_count']].head(10))

# Contar valores nulos para evaluar el éxito del procesamiento
print("Valores nulos en las columnas derivadas:")
print(purchase_prices[['size_ml', 'pack_count']].isnull().sum())

# Identificar valores problemáticos
problematic_rows = purchase_prices[purchase_prices['size_ml'].isnull()]
print("Valores problemáticos en la columna 'Size':")
print(problematic_rows['Size'].unique())


Primeras filas del DataFrame después de procesar la columna 'Size':
     Size  size_ml  pack_count
0   750ml    750.0         1.0
1   750ml    750.0         1.0
2   750ml    750.0         1.0
3   750ml    750.0         1.0
4   750ml    750.0         1.0
5   750ml    750.0         1.0
6   750ml    750.0         1.0
7  1000ml   1000.0         1.0
8  1000ml   1000.0         1.0
9   750ml    750.0         1.0
Valores nulos en las columnas derivadas:
size_ml       5
pack_count    5
dtype: int64
Valores problemáticos en la columna 'Size':
<StringArray>
['unknown', <NA>]
Length: 2, dtype: string


In [205]:
# Mostrar los valores únicos para 'Size', 'size_ml', y 'pack_count'
for column in ['Size', 'size_ml', 'pack_count']:
    print(f"Valores únicos en la columna '{column}':")
    print(purchase_prices[column].unique())
    print()  


Valores únicos en la columna 'Size':
<StringArray>
[      '750ml',      '1000ml',      '1750ml',        '50ml',       '375ml',
  '100ml 4 pk',   '50ml 5 pk',       '100ml',       '200ml',   '50ml 4 pk',
   '50ml 3 pk',       '300ml',  '200ml 4 pk',     'unknown',  '750ml 2 pk',
  '250ml 4 pk',      '1500ml',      '3000ml',      '5000ml',      '4000ml',
  '187ml 4 pk',       '150ml',       '187ml',       '500ml',       '720ml',
       '650ml',  '200ml 3 pk',  '187ml 3 pk',       '330ml',       '250ml',
  '750ml + 3/',     '18000ml',       '180ml',  '750ml + 4/',      '6000ml',
 '1000ml 2 pk',  '750ml 12 p',  '750ml 6 pk',   '20ml 5 pk',          <NA>,
  '375ml 2 pk',     '20000ml',  '50ml 12 pk',  '750ml 3 pk',  '375ml 3 pk',
  '750ml + 1/',  '200ml 5 pk',     '162.5ml',       '400ml',      '1100ml',
       '600ml',     '19500ml',       '560ml',      '3750ml',     '750ml 3',
      '9000ml']
Length: 56, dtype: string

Valores únicos en la columna 'size_ml':
[  750.   1000.   1750.     50

In [206]:
# Calcular el tamaño total del paquete
purchase_prices['Total_size_ml'] = purchase_prices['size_ml'] * purchase_prices['pack_count']

# Verificar los resultados
print(purchase_prices[['Size', 'size_ml', 'pack_count', 'Total_size_ml']].head(10))


     Size  size_ml  pack_count  Total_size_ml
0   750ml    750.0         1.0          750.0
1   750ml    750.0         1.0          750.0
2   750ml    750.0         1.0          750.0
3   750ml    750.0         1.0          750.0
4   750ml    750.0         1.0          750.0
5   750ml    750.0         1.0          750.0
6   750ml    750.0         1.0          750.0
7  1000ml   1000.0         1.0         1000.0
8  1000ml   1000.0         1.0         1000.0
9   750ml    750.0         1.0          750.0


In [207]:
print("PurchasePrices Info:")
print(purchase_prices.info())

PurchasePrices Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12261 entries, 0 to 12260
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12261 non-null  Int64  
 1   Description     12260 non-null  string 
 2   Price           12261 non-null  Float64
 3   Size            12260 non-null  string 
 4   Volume          12256 non-null  float64
 5   Classification  12261 non-null  Int64  
 6   PurchasePrice   12261 non-null  Float64
 7   VendorNumber    12261 non-null  Int64  
 8   VendorName      12261 non-null  string 
 9   size_ml         12256 non-null  float64
 10  pack_count      12256 non-null  float64
 11  Total_size_ml   12256 non-null  float64
dtypes: Float64(2), Int64(3), float64(4), string(3)
memory usage: 1.2 MB
None


#### Elimino la columna Volume porque ya fue reemplazada por size_ml (primero me fijo si tienen los mismos valores)

In [208]:
# Reemplazar valores "Unknown" por NaN 
purchase_prices['Volume'] = purchase_prices['Volume'].replace("Unknown", np.nan)

# Verificar si size_ml es igual a Volume (considerando valores nulos)
comparison = purchase_prices['size_ml'] == purchase_prices['Volume']

# Contar cuántos valores son iguales y diferentes
print("Comparación entre size_ml y Volume:")
print(f"Iguales: {comparison.sum()}")
print(f"Diferentes o NaN: {len(comparison) - comparison.sum()}")

# Identificar diferencias reales (excluyendo NaN)
different_rows = purchase_prices[~comparison & ~(purchase_prices['size_ml'].isnull() & purchase_prices['Volume'].isnull())]

# Verificar si los valores diferentes son únicamente NaN
nan_differences = purchase_prices[(purchase_prices['size_ml'].isnull()) & (purchase_prices['Volume'].isnull())]

# Si todos los valores diferentes son NaN o no hay diferencias, eliminar la columna
if len(different_rows) == 0:
    purchase_prices.drop(columns=['Volume'], inplace=True)
    print("La columna 'Volume' fue eliminada porque es igual a 'size_ml', considerando NaN como iguales.")
else:
    print("La columna 'Volume' no fue eliminada porque contiene valores diferentes a 'size_ml' que no son NaN.")

# Verificar el resultado final
print(purchase_prices.info())


Comparación entre size_ml y Volume:
Iguales: 12256
Diferentes o NaN: 5
La columna 'Volume' fue eliminada porque es igual a 'size_ml', considerando NaN como iguales.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12261 entries, 0 to 12260
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12261 non-null  Int64  
 1   Description     12260 non-null  string 
 2   Price           12261 non-null  Float64
 3   Size            12260 non-null  string 
 4   Classification  12261 non-null  Int64  
 5   PurchasePrice   12261 non-null  Float64
 6   VendorNumber    12261 non-null  Int64  
 7   VendorName      12261 non-null  string 
 8   size_ml         12256 non-null  float64
 9   pack_count      12256 non-null  float64
 10  Total_size_ml   12256 non-null  float64
dtypes: Float64(2), Int64(3), float64(3), string(3)
memory usage: 1.1 MB
None


In [209]:
# Unificar todos los floats a un único formato (float64)
float_columns = purchase_prices.select_dtypes(include=['Float64', 'float64']).columns
purchase_prices[float_columns] = purchase_prices[float_columns].astype('float64')

# Verificar que los cambios se aplicaron correctamente
print("Tipos de datos después de unificar formatos de float:")
print(purchase_prices.dtypes)


Tipos de datos después de unificar formatos de float:
Brand                      Int64
Description       string[python]
Price                    float64
Size              string[python]
Classification             Int64
PurchasePrice            float64
VendorNumber               Int64
VendorName        string[python]
size_ml                  float64
pack_count               float64
Total_size_ml            float64
dtype: object


In [210]:
# Filtrar las filas con valores NaN o nulos en cualquier columna
nan_rows = purchase_prices[purchase_prices.isnull().any(axis=1)]

# Mostrar las filas con valores NaN o nulos
print("Filas con valores NaN o nulos:")
print(nan_rows)


Filas con valores NaN o nulos:
       Brand                   Description  Price     Size  Classification  \
542     2993             angostura bitters   7.49  unknown               1   
5921    9908      tito's copper mug 2 pack  21.01  unknown               1   
7915    4202                          <NA>   0.00     <NA>               1   
8795    8992                      group 92   1.99  unknown               1   
10009  90590  overture champagne 2glass pk  19.95  unknown               2   

       PurchasePrice  VendorNumber             VendorName  size_ml  \
542             5.39          5895  mizkan americas, inc.      NaN   
5921           16.15          4425  martignetti companies      NaN   
7915           11.19           480        bacardi usa inc      NaN   
8795            1.43          1703   alisa carr beverages      NaN   
10009          13.12          4425  martignetti companies      NaN   

       pack_count  Total_size_ml  
542           NaN            NaN  
5921     

borramos los valores de nan directamente? tienen de todas maneras datos pero si nos importa la cantidad que se vende, en esas filas no hay datos.

In [211]:
# Verificar si hay duplicados
print("\n¿Hay duplicados en los datos?")
print(purchase_prices.duplicated().any())

# Mostrar filas duplicadas, si existen
if purchase_prices.duplicated().any():
    print("\nFilas duplicadas detectadas:")
    display(purchase_prices[purchase_prices.duplicated()])

# Eliminar duplicados
data = purchase_prices.drop_duplicates()

# Verificar nuevamente
print("\nDatos después de eliminar duplicados:")
display(purchase_prices.head())


¿Hay duplicados en los datos?
False

Datos después de eliminar duplicados:


,Brand,Description,Price,Size,Classification,PurchasePrice,VendorNumber,VendorName,size_ml,pack_count,Total_size_ml
0,58,gekkeikan black & gold sake,12.99,750ml,1,9.28,8320,shaw ross int l imp ltd,750.0,1.0,750.0
1,62,herradura silver tequila,36.99,750ml,1,28.67,1128,brown-forman corp,750.0,1.0,750.0
2,63,herradura reposado tequila,38.99,750ml,1,30.46,1128,brown-forman corp,750.0,1.0,750.0
3,72,no. 3 london dry gin,34.99,750ml,1,26.11,9165,ultra beverage company llp,750.0,1.0,750.0
4,75,three olives tomato vodka,14.99,750ml,1,10.94,7245,proximo spirits inc.,750.0,1.0,750.0


#### Crear clave primaria supplier_id

In [215]:
# Verificar unicidad en la combinación de VendorNumber y Brand
if purchase_prices.duplicated(subset=['VendorNumber', 'Brand']).any():
    print("Advertencia: Existen duplicados en la combinación de 'VendorNumber' y 'Brand'.")
else:
    print("No se encontraron duplicados en la combinación de 'VendorNumber' y 'Brand'.")

# Crear la columna 'SupplierID' como un índice único basado en la combinación de VendorNumber y Brand
purchase_prices['SupplierID'] = (
    purchase_prices.groupby(['VendorNumber', 'Brand']).ngroup() + 1
)

# Establecer 'SupplierID' como índice
purchase_prices.set_index('SupplierID', inplace=True)

# Guardar el DataFrame actualizado
purchase_prices.to_csv('purchase_prices.csv')

# Visualizar las primeras filas para confirmar los cambios
print(purchase_prices.head())


No se encontraron duplicados en la combinación de 'VendorNumber' y 'Brand'.
            Brand                  Description  Price   Size  Classification  \
SupplierID                                                                     
6873           58  gekkeikan black & gold sake  12.99  750ml               1   
702            62     herradura silver tequila  36.99  750ml               1   
703            63   herradura reposado tequila  38.99  750ml               1   
7359           72         no. 3 london dry gin  34.99  750ml               1   
6376           75    three olives tomato vodka  14.99  750ml               1   

            PurchasePrice  VendorNumber                  VendorName  size_ml  \
SupplierID                                                                     
6873                 9.28          8320     shaw ross int l imp ltd    750.0   
702                 28.67          1128           brown-forman corp    750.0   
703                 30.46          1128    

In [216]:
purchase_prices.head()

,Brand,Description,Price,Size,Classification,PurchasePrice,VendorNumber,VendorName,size_ml,pack_count,Total_size_ml
SupplierID,,,,,,,,,,,
6873,58,gekkeikan black & gold sake,12.99,750ml,1,9.28,8320,shaw ross int l imp ltd,750.0,1.0,750.0
702,62,herradura silver tequila,36.99,750ml,1,28.67,1128,brown-forman corp,750.0,1.0,750.0
703,63,herradura reposado tequila,38.99,750ml,1,30.46,1128,brown-forman corp,750.0,1.0,750.0
7359,72,no. 3 london dry gin,34.99,750ml,1,26.11,9165,ultra beverage company llp,750.0,1.0,750.0
6376,75,three olives tomato vodka,14.99,750ml,1,10.94,7245,proximo spirits inc.,750.0,1.0,750.0


### Analisis de InvoicePurchases

In [218]:
invoice_purchases = pd.read_csv('InvoicePurchases.csv')
invoice_purchases.head()

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2016-01-04,8124,2015-12-21,2016-02-16,6,214.26,3.47,NaN
1,4466,AMERICAN VINTAGE BEVERAGE,2016-01-07,8137,2015-12-22,2016-02-21,15,140.55,8.57,NaN
2,388,ATLANTIC IMPORTING COMPANY,2016-01-09,8169,2015-12-24,2016-02-16,5,106.60,4.61,NaN
3,480,BACARDI USA INC,2016-01-12,8106,2015-12-20,2016-02-05,10100,137483.78,2935.20,NaN
4,516,BANFI PRODUCTS CORP,2016-01-07,8170,2015-12-24,2016-02-12,1935,15527.25,429.20,NaN


In [219]:
# Revisar los tipos de datos para cada tabla
invoice_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   VendorNumber  5543 non-null   int64  
 1   VendorName    5543 non-null   object 
 2   InvoiceDate   5543 non-null   object 
 3   PONumber      5543 non-null   int64  
 4   PODate        5543 non-null   object 
 5   PayDate       5543 non-null   object 
 6   Quantity      5543 non-null   int64  
 7   Dollars       5543 non-null   float64
 8   Freight       5543 non-null   float64
 9   Approval      374 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 433.2+ KB


#### que tiene la columna approval?

In [221]:
invoice_purchases['Approval'].unique()

array([nan, 'Frank Delahunt'], dtype=object)

#### La elimino porque no hay un solo nombre

In [ ]:
invoice_purchases = invoice_purchases.drop(columns=['Approval'])

In [223]:
# Convertir columnas de tipo object a string
for column in invoice_purchases.select_dtypes(include=['object']).columns:
    invoice_purchases[column] = invoice_purchases[column].astype('string')

# Convertir columnas a tipos de datos apropiados
invoice_purchases['VendorNumber'] = invoice_purchases['VendorNumber'].astype('int64')
invoice_purchases['PONumber'] = invoice_purchases['PONumber'].astype('int64')
invoice_purchases['Quantity'] = invoice_purchases['Quantity'].astype('int64')
invoice_purchases['Dollars'] = invoice_purchases['Dollars'].astype('float64')
invoice_purchases['Freight'] = invoice_purchases['Freight'].astype('float64')

# Convertir columnas de fecha a datetime
invoice_purchases['InvoiceDate'] = pd.to_datetime(invoice_purchases['InvoiceDate'], errors='coerce')
invoice_purchases['PODate'] = pd.to_datetime(invoice_purchases['PODate'], errors='coerce')
invoice_purchases['PayDate'] = pd.to_datetime(invoice_purchases['PayDate'], errors='coerce')

# Verificar cambios
print(invoice_purchases.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   VendorNumber  5543 non-null   int64         
 1   VendorName    5543 non-null   string        
 2   InvoiceDate   5543 non-null   datetime64[ns]
 3   PONumber      5543 non-null   int64         
 4   PODate        5543 non-null   datetime64[ns]
 5   PayDate       5543 non-null   datetime64[ns]
 6   Quantity      5543 non-null   int64         
 7   Dollars       5543 non-null   float64       
 8   Freight       5543 non-null   float64       
dtypes: datetime64[ns](3), float64(2), int64(3), string(1)
memory usage: 389.9 KB
None


In [225]:
invoice_purchases.head()

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight
0,105,ALTAMAR BRANDS LLC,2016-01-04,8124,2015-12-21,2016-02-16,6,214.26,3.47
1,4466,AMERICAN VINTAGE BEVERAGE,2016-01-07,8137,2015-12-22,2016-02-21,15,140.55,8.57
2,388,ATLANTIC IMPORTING COMPANY,2016-01-09,8169,2015-12-24,2016-02-16,5,106.60,4.61
3,480,BACARDI USA INC,2016-01-12,8106,2015-12-20,2016-02-05,10100,137483.78,2935.20
4,516,BANFI PRODUCTS CORP,2016-01-07,8170,2015-12-24,2016-02-12,1935,15527.25,429.20
